In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
    'sort': 'last_login',
}


In [27]:
innmeldingar = pd.read_csv("enrollments.csv")
utan_sis = pd.read_csv("utan_sis.csv")

In [28]:
innmeldingar.head()

,value.user_id,value.course_id,value.created_at,value.workflow_state,value.updated_at,value.last_activity_at,value.type
0,67355,25541,2023-12-17T02:29:49.872Z,active,2024-09-24T09:56:44.315Z,2025-04-27T00:06:58.653Z,StudentEnrollment
1,47509,25834,2024-01-07T07:42:53.679Z,active,2024-09-24T10:00:03.357Z,2025-05-06T09:59:05.654Z,StudentEnrollment
2,94554,25541,2023-12-19T16:12:55.612Z,active,2024-09-24T09:40:55.958Z,2025-03-29T12:51:13.443Z,StudentEnrollment
3,79426,24602,2023-12-15T06:12:21.014Z,active,2024-09-24T09:45:33.084Z,2024-10-28T11:18:08.898Z,StudentEnrollment
4,83726,25881,2023-12-16T16:14:38.638Z,active,2024-09-24T09:43:09.498Z,2024-11-10T08:58:44.358Z,StudentEnrollment


In [29]:
utan_sis.head()

,id,name,sis_user_id,created_at
0,1206,Katrine Abelsen,NaN,2017-11-06T13:36:18+01:00
1,1207,Enok Abrahamsen,NaN,2017-11-06T13:36:38+01:00
2,27329,ACS,NaN,2018-10-15T15:45:01+02:00
3,102310000000076186,Sumaya Mohammed Adan,NaN,2021-07-27T03:07:42+02:00
4,12788,Najma ahmed,NaN,2018-04-23T22:08:08+02:00


## 1. Innmeldt i emne
No kan eg lage ei liste over brukarar som ikkje har sis_id, men som er meldt inn i eit emne:

In [35]:
innmeldingar_utan_sis = []
for i, r in utan_sis.iterrows():
    if r['id'] in innmeldingar['value.user_id'].values:
        innmeldingar_utan_sis.append(r['id'])

Denne lista må eg faktisk gå gjennom manuelt for å slette frå Canvas. (Å slette ved sis_import krev sis_id, som vi ikkje har.)

## 2. Ikkje innmeldt i emne
Kan eg også lage ei liste over dei utan sis_id som *ikkje* er innmeldt i emne?

In [38]:
ingen_innmeldingar = []
for i, r in utan_sis.iterrows():
    if r['id'] not in innmeldingar['value.user_id'].values:
        ingen_innmeldingar.append(r['id'])

In [39]:
ingen_innmeldingar

[1206,
 1207,
 102310000000076186,
 12788,
 4361,
 13013,
 40167,
 1255,
 1208,
 1209,
 29612,
 27730,
 28730,
 77712,
 3862,
 4000,
 52757,
 27233,
 27688,
 38868,
 74,
 26902,
 26901,
 102310000000086688,
 28497,
 13170,
 5250,
 54879,
 12389,
 12997,
 29269,
 29746,
 39894,
 27451,
 1211,
 26771,
 1210,
 210,
 1126,
 22037,
 22426,
 54884,
 1212,
 1213,
 12421,
 1214,
 12416,
 12343,
 211,
 38839,
 54991,
 27501,
 27502,
 12387,
 12417,
 41379,
 54793,
 12477,
 36031,
 12392,
 27591,
 28295,
 27331,
 54956,
 55070,
 55074,
 5232,
 54959,
 4102,
 26301,
 12469,
 30254,
 3416,
 27711,
 39678,
 12420,
 12405,
 40814,
 3992,
 1257,
 39366,
 39605,
 12789,
 39858,
 39859,
 39413,
 1258,
 408,
 1215,
 54782,
 54805,
 55391,
 29858,
 27696,
 41119,
 54990,
 12385,
 27402,
 27712,
 28488,
 28496,
 12393,
 411,
 1216,
 1278,
 12383,
 1060,
 212,
 12403,
 102310000000065006,
 13067,
 1217,
 4136,
 12364,
 30143,
 1218,
 12370,
 102310000000030244,
 3585,
 27091,
 607,
 1219,
 54945,
 4156,
 3

Her er noko galt; brukar 27730 (Torill Andersen, kanskje innmeldt medan vi framleis hadde eigenpåmelding aktivt?) ligg ikkje i lista over enrollments, men ho er meldt inn i emne 215 (Praksisrettleiarkurset). Eg undersøker først om ho kan bli henta ut via GrapQL, og det kan ho. 


In [43]:
innmeldingar.loc[innmeldingar['value.user_id'] == 27730]

,value.user_id,value.course_id,value.created_at,value.workflow_state,value.updated_at,value.last_activity_at,value.type


In [44]:
utan_sis.loc[utan_sis['id'] == 27730]

,id,name,sis_user_id,created_at
14,27730,Torill Andersen,NaN,2018-12-03T09:24:22+01:00


Så konklusjonen må bli at eg ikkje får med meg alle sjølv om eg bruker CD2 (for den viser berre 30 dagar tilbake i tid)? Men eg prøver allikevel.